#Segmenting and Clustering Neighborhoods in Jackson, Ms

Table of Contents
1. Download and Explore Sales Tax Dataset 
2. Explore Neighborhoods in Jackson, MS 
3. Analyze Each Neighborhood 
4. Cluster Neighborhoods 
5. Examine Clusters to determine optimal homebuying location

This data analysis was established to help young couples with a median income of $109,000 identify the optimal neighborhood within Jackson, MS to purchase their first home.  Critical factors are home prices, sales taxes and promixity to amenities.  

Import necessary Libraries

In [1]:
import random # library for random number generation
import numpy as np # library for vectorized computation
import pandas as pd # library to process data as dataframes

In [2]:
import json # library to handle JSON files

Dataset abstract:  Sales tax data is captured directly from the MS Department of Revenue and specific to the City of Jackson. It is compiled from Gross Sales Tax reported by taxpayers each month. Gross Sales Tax...

In [3]:
!wget -q -O 'jackson_data.json'  https://data.jacksonms.gov/api/views/d36h-vd4y/rows.json?accessType=DOWNLOAD
print('Data downloaded!')

Data downloaded!


In [4]:
with open('jackson_data.json') as json_data:
    jackson_data = json.load(json_data)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

A careful review of the site where the data was contained revealed the following; 

{
  "code" : "not_found",
  "error" : true,
  "message" : "Cannot find web property with name data.jacksonms.gov"
}

As result, I am unable to proceed as originally planned using gross tax sales as a cross comparison to identify areas with low sales tax collections as a determination along with venue proximity for selecting the best neighborhood within the Jackson, Ms area.  I will shift my analysis to exploring housing data extracted from (https:zillow/data/research). 

Home Listings and Sales

Zillow provides data on sold homes, including median sale price for various housing types, sale counts (for which there’s detailed methodology), and foreclosures provided as a share of all sales in which the home was previously foreclosed upon. There is also current and historical for-sale listings data, ranging from median list prices and inventory counts to share of listings with a price cut, median price cut size, age of inventory, and the days a listing spent on Zillow before the sale was final. Inventory and other housing data also are available for local markets.

In [6]:
neighborhoods_subset = pd.read_csv('https://raw.githubusercontent.com/rpjones31/Capstone-Notebook/master/Neighborhood_Price.csv')
neighborhoods_subset.head()

,RegionName,City,State,Metro,CountyName,SizeRank,2010-01,2010-02,2010-03,2010-04,...,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05
0,Northeast Dallas,Dallas,TX,Dallas-Fort Worth-Arlington,Dallas County,1,229000.0,224900.0,233500.0,230000.0,...,389000.0,390000.0,398500.0,399000.0,390000.0,387500.0,399000.0,399000.0,399000.0,399250.0
1,Maryvale,Phoenix,AZ,Phoenix-Mesa-Scottsdale,Maricopa County,2,NaN,NaN,NaN,NaN,...,184800.0,184950.0,185000.0,185000.0,185000.0,185000.0,186719.0,190000.0,192950.0,194500.0
2,Paradise,Las Vegas,NV,Las Vegas-Henderson-Paradise,Clark County,3,145900.0,149000.0,144900.0,138900.0,...,275000.0,272272.0,270000.0,274000.0,273499.5,275000.0,272000.0,269893.5,270000.0,275000.0
3,Upper West Side,New York,NY,New York-Newark-Jersey City,New York County,4,NaN,NaN,NaN,NaN,...,1795000.0,1750000.0,1795000.0,1797000.0,1875000.0,1900000.0,1858500.0,1800000.0,1800000.0,1795000.0
4,South Los Angeles,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles County,5,200000.0,205000.0,205000.0,210000.0,...,485000.0,490000.0,490000.0,480000.0,489900.0,479999.0,475000.0,490000.0,499000.0,499000.0


In [12]:
neighborhoods_subset=neighborhoods_subset[neighborhoods_subset.City=='Jackson']
neighborhoods_subset.head()

,RegionName,City,State,Metro,CountyName,SizeRank,2010-01,2010-02,2010-03,2010-04,...,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05
1789,Greater Belhaven,Jackson,MS,Jackson,Hinds County,1790,NaN,NaN,NaN,NaN,...,225000.0,225000.0,225000.0,225250.0,229750.0,234000.0,229900.0,225000.0,212500.0,194900.0
1964,Fondren North,Jackson,MS,Jackson,Hinds County,1965,NaN,NaN,NaN,NaN,...,249700.0,245000.0,246500.0,246500.0,249900.0,259632.0,250000.0,249000.0,249900.0,244900.0
2134,Fondern-Cherokee Heights,Jackson,MS,Jackson,Hinds County,2135,NaN,NaN,NaN,NaN,...,166250.0,168250.0,175449.5,176499.5,176000.0,175999.0,177000.0,190749.5,191700.0,192450.0


In order to determine proximity to venues a breakdown of the three regions will be reviewed.  Co-ordinates or latitude & longitude will be obtained by passing the RegionName value through geocoding. 

In [13]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [17]:
# Select Belhaven Neighborhood

address = 'Belhaven, Jackson, MS'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Latitude and longitude of '+address+' are {}, {}.'.format(latitude, longitude))

Latitude and longitude of Belhaven, Jackson, MS are 32.3201811, -90.1690766468281.


#Now that neighborhood data has been retrieved for Belhaven, Foursquare Location will be used to get venue lists. Veue lists can be retrieved using search endpoint. 

In [19]:
CLIENT_ID = 'UBUOJWVJ5TJTETERBT15ZDGP4PRKXDCWCGM21SCEKTD3IOYJ' # your Foursquare ID
CLIENT_SECRET = '411YIZ1O5BQ0AXIQ01BLQTN3TIS1TIICPGT5T3MT4CJUDY4Z' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + 'UBUOJWVJ5TJTETERBT15ZDGP4PRKXDCWCGM21SCEKTD3IOYJ')
print('CLIENT_SECRET: ' + '411YIZ1O5BQ0AXIQ01BLQTN3TIS1TIICPGT5T3MT4CJUDY4Z')

Your credentails:
CLIENT_ID: UBUOJWVJ5TJTETERBT15ZDGP4PRKXDCWCGM21SCEKTD3IOYJ
CLIENT_SECRET: 411YIZ1O5BQ0AXIQ01BLQTN3TIS1TIICPGT5T3MT4CJUDY4Z


In [80]:
search_query = 'Venues'
radius=500                              # radius in meters

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)

In [81]:
#Send the GET request and examine the resutls
import requests 
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d30fee4e706550025cd011e'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Jackson',
  'headerFullLocation': 'Jackson',
  'headerLocationGranularity': 'city',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 32.3246811045, 'lng': -90.16376160367851},
   'sw': {'lat': 32.3156810955, 'lng': -90.1743916899777}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e9b4d19d3e3697fc02e29fe',
       'name': 'Lake Belhaven',
       'location': {'lat': 32.320531213579216,
        'lng': -90.16752963399975,
        'labeledLatLngs': [{'label': 'display',
          'lat': 32.320531213579216,
          'lng': -90.16752963399975}],
        'distance': 

In [83]:
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

In [99]:
# get relevant part of JSON and transform it into a pandas dataframe:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)
nearby_venues.head()

,reasons.count,reasons.items,referralId,venue.categories,venue.id,venue.location.address,venue.location.cc,venue.location.city,venue.location.country,venue.location.crossStreet,venue.location.distance,venue.location.formattedAddress,venue.location.labeledLatLngs,venue.location.lat,venue.location.lng,venue.location.postalCode,venue.location.state,venue.name,venue.photos.count,venue.photos.groups
0,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4e9b4d19d3e3697fc02e29fe-0,"[{'id': '4bf58dd8d48988d161941735', 'name': 'L...",4e9b4d19d3e3697fc02e29fe,NaN,US,Jackson,United States,NaN,150,"[Jackson, MS, United States]","[{'label': 'display', 'lat': 32.32053121357921...",32.320531,-90.167530,NaN,MS,Lake Belhaven,0,[]
1,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4c899f3152a98cfa46d125e9-1,"[{'id': '4bf58dd8d48988d165941735', 'name': 'S...",4c899f3152a98cfa46d125e9,NaN,US,Jackson,United States,NaN,147,"[Jackson, MS 39202, United States]","[{'label': 'display', 'lat': 32.32125, 'lng': ...",32.321250,-90.170011,39202,MS,Belhaven Bowl,0,[]
2,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4bf45932e5eba593338d1f90-2,"[{'id': '4bf58dd8d48988d181941735', 'name': 'M...",4bf45932e5eba593338d1f90,1119 Pinehurst Pl,US,Jackson,United States,btwn Peachtree St & Olive St,365,[1119 Pinehurst Pl (btwn Peachtree St & Olive ...,"[{'label': 'display', 'lat': 32.317073, 'lng':...",32.317073,-90.170314,39202,MS,Eudora Welty House,0,[]
3,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4b523005f964a520506e27e3-3,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",4b523005f964a520506e27e3,1841 Laurel St,US,Jackson,United States,btwn Howard St & Piedmont St,485,[1841 Laurel St (btwn Howard St & Piedmont St)...,"[{'label': 'display', 'lat': 32.31816290964937...",32.318163,-90.164505,39202,MS,Laurel Street Park,0,[]


In [104]:
# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

nearby_venues.head()

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Lake Belhaven,"[{'id': '4bf58dd8d48988d161941735', 'name': 'L...",32.320531,-90.167530
1,Belhaven Bowl,"[{'id': '4bf58dd8d48988d165941735', 'name': 'S...",32.321250,-90.170011
2,Eudora Welty House,"[{'id': '4bf58dd8d48988d181941735', 'name': 'M...",32.317073,-90.170314
3,Laurel Street Park,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P...",32.318163,-90.164505


Belhaven Results:  only 4 venues within 500 meters were returned by Foursquare. 

In [105]:
address = 'Fondren, MS'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Latitude and longitude of '+address+' are {}, {}.'.format(latitude, longitude))

Latitude and longitude of Fondren, MS are 33.3534553, -90.358419.


In [107]:
search_query = 'Venues'
radius=500                              # radius in meters

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)

In [108]:
#Send the GET request and examine the resutls
import requests 
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d310cf2bf7dde002c58dc34'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Current map view',
  'headerFullLocation': 'Current map view',
  'headerLocationGranularity': 'unknown',
  'totalResults': 0,
  'suggestedBounds': {'ne': {'lat': 33.3579553045, 'lng': -90.35304173366316},
   'sw': {'lat': 33.3489552955, 'lng': -90.36379626633683}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': []}]}}

In [109]:
# get relevant part of JSON and transform it into a pandas dataframe:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)
nearby_venues.head()

""


Fondren Results: 0 venues within 500 meters were returned by Foursquare.

In [110]:
address = 'Cherokee Heights, MS'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Latitude and longitude of '+address+' are {}, {}.'.format(latitude, longitude))

Latitude and longitude of Cherokee Heights, MS are 33.4762328, -88.827832.


In [111]:
search_query = 'Venues'
radius=500                              # radius in meters

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)

In [113]:
#Send the GET request and examine the resutls
import requests 
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d310db6531593002cc2ee72'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Starkville',
  'headerFullLocation': 'Starkville',
  'headerLocationGranularity': 'city',
  'totalResults': 0,
  'suggestedBounds': {'ne': {'lat': 33.4807328045, 'lng': -88.82244712611761},
   'sw': {'lat': 33.471732795499996, 'lng': -88.83321687388239}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': []}]}}

In [114]:
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)
nearby_venues.head()

""


Cherokee Heights Results: 0 venues within 500 meters were returned by Foursquare.

Approach:  Conduct a comparison of three neighborhoods within Jackson area with relevant homes prices for potential homeowners with a median income of $109,000
Results:  Greater Belhaven returned highest number of venues and should thereby be considered as the optimal location 